In [28]:
from entropy_compressor import ZCompressor
from quantizator import Quantizator
from wavelet_compressor import WaveletCompressor

import os
import numpy as np
from typing import Dict, List
from pathlib import Path
from pprint import pprint

COMPRESS_PATH = 'compressed_low'
DECOMPRESS_PATH = 'decompressed'
IMAGE_NAME = 'low.png'
MESH_KEYS = ['faces', 'neighbour', 'owner', 'points']
PTURHO_KEYS = ['p', 'T', 'U', 'rho']

class Decompressor:
    def __init__(self):
        if not os.path.exists(DECOMPRESS_PATH):
            os.mkdir(DECOMPRESS_PATH)
        self.entropy_compressor = ZCompressor()
        self.quantizator = Quantizator()
        self.wavelet_compressor = WaveletCompressor()
    
    def total_decompress(self, compressed_data: str, q_step: np.float64, min_val: np.float64) -> np.ndarray:
        z_decoded = self.entropy_compressor.decompress(bytes.fromhex(compressed_data))
        print('z decoded')
        dequantized = self.quantizator.dequantize(z_decoded, q_step, min_val)
        print('dequantized')
        decompressed = self.wavelet_compressor.inverse_wavelet_transform(dequantized)

        return decompressed

    def load_to_file(self, data: List[np.ndarray | np.float64], file_path: str, header_data: dict[str]) -> None:
        z_compressed, q_step, min_val = data
        decompressed = self.total_decompress(z_compressed, q_step, min_val)
        
    def is_two_floats(self, line: str) -> bool:
        try:
            a, b = line.split()
            a = float(a)
            b = float(b)
        except:
            return False
        return True
    
    def read_mesh_file(self, file_path: str) -> Dict[str, List[np.ndarray | np.float64]]:
        data = {}
        for key in MESH_KEYS:
            data[key] = None
        i = 0
        with open(file_path, mode='r') as f:
            dictionary = ''
            quantilization_keys = f.readline().strip()
            while True:
                z_compressed = f.readline().strip()
                thirdline = f.readline().strip()

                dictionary += thirdline

                while not self.is_two_floats(thirdline):
                    dictionary += thirdline
                    thirdline = f.readline().strip()
                
                quantilization_keys = thirdline
                dictionary = eval(dictionary)

                if not quantilization_keys or not z_compressed:
                    break
            
                q_step, min_val = map(float, quantilization_keys.split())
                data[MESH_KEYS[i]] = [z_compressed, q_step, min_val, dictionary]
                i += 1
        return data
    
    def read_pturho_file(self, file_path: str) -> Dict[str, List[np.ndarray | np.float64]]:
        data = {}
        i = 0
        with open(file_path, mode='r') as f:
            times = f.readline().strip().split(';')
            for key in times:
                data[key] = None
            while True:
                quantilization_keys_p = f.readline().strip()
                z_compressed_p = f.readline().strip()
                details_p = eval(f.readline().strip())

                quantilization_keys_t = f.readline().strip()
                z_compressed_t = f.readline().strip()
                details_t = eval(f.readline().strip())

                quantilization_keys_u = f.readline().strip()
                z_compressed_u = f.readline().strip()
                details_u = eval(f.readline().strip())

                quantilization_keys_rho = f.readline().strip()
                z_compressed_rho = f.readline().strip()
                details_rho = eval(f.readline().strip())

                # Check if either line is empty, indicating the end of the file
                if not quantilization_keys_p or not z_compressed_p:
                    break
                
                q_step_p, min_val_p = map(float, quantilization_keys_p.split())
                q_step_t, min_val_t = map(float, quantilization_keys_t.split())
                q_step_u, min_val_u = map(float, quantilization_keys_u.split())
                q_step_rho, min_val_rho = map(float, quantilization_keys_rho.split())
                data[times[i]] = [
                    [z_compressed_p, q_step_p, min_val_p, details_p],
                    [z_compressed_t, q_step_t, min_val_t, details_t],
                    [z_compressed_u, q_step_u, min_val_u, details_u],
                    [z_compressed_rho, q_step_rho, min_val_rho, details_rho],
                ]
                i += 1
        return data

In [29]:
decompressor = Decompressor()
mesh_data = decompressor.read_mesh_file('compressed/mesh')
print('read')
decompressor.total_decompress(*mesh_data['faces'])

SyntaxError: '[' was never closed (<string>, line 1)

In [ ]:
[{'ad': array([[-2.03125000e+01,  6.10625000e+01,  6.68750000e+00,[{'ad': array([[-2.03125000e+01,  6.10625000e+01,  6.68750000e+00,-4.74375000e+01],[-2.72851562e+01,  6.63945312e+01,  1.94023438e+01,-5.85117188e+01],[-2.13085937e+01,  6.18242187e+01,  8.50390625e+00,-4.90195312e+01],[-3.09375000e+01,  6.91875000e+01,  2.60625000e+01,-6.43125000e+01],[-2.26367188e+01,  6.28398438e+01,  1.09257813e+01,-5.11289063e+01],[-2.72851563e+01,  6.63945313e+01,  1.94023438e+01,-5.85117188e+01],[-2.26367188e+01,  6.28398437e+01,  1.09257812e+01,-5.11289062e+01],[-2.46289062e+01,  6.43632812e+01,  1.45585938e+01,-5.42929688e+01],[-3.09375000e+01,  6.91875000e+01,  2.60625000e+01,-6.43125000e+01],[-2.26367187e+01,  6.28398437e+01,  1.09257813e+01,-5.11289062e+01],[-2.79492187e+01,  6.69023438e+01,  2.06132812e+01,-5.95664062e+01],[-1.77578125e+01,  5.96796875e+01,  1.86718750e+00,-4.37890625e+01],[-1.81054688e+01,  7.43554688e+01, -5.79296875e+00,-5.04570313e+01],[-4.81601562e+01,  1.23941406e+02,  3.12773438e+01,-1.07058594e+02],[-4.26640625e+01,  1.09210938e+02,  2.69609375e+01,-9.35078125e+01],[-4.26250000e+01,  1.08125000e+02,  2.73750000e+01,-9.28750000e+01],[-4.63710937e+01,  1.08667969e+02,  3.53164062e+01,-9.76132812e+01],[-4.35000000e+01,  1.26750000e+02,  2.05000000e+01,-1.03750000e+02],[-4.32851563e+01,  1.10863281e+02,  2.75273438e+01,-9.51054688e+01],[-4.26250000e+01,  1.08125000e+02,  2.73750000e+01,-9.28750000e+01],[-4.31757812e+01,  1.08535156e+02,  2.83867187e+01,-9.37460937e+01],[-4.20625000e+01,  9.81875000e+01,  3.09375000e+01,-8.70625000e+01],[-4.87304688e+01,  1.50746094e+02,  2.02070313e+01,-1.22222656e+02],[-4.20625000e+01,  9.81875000e+01,  3.09375000e+01,-8.70625000e+01],[-4.26250000e+01,  1.08125000e+02,  2.73750000e+01,-9.28750000e+01],[-4.09726563e+01,  1.06894531e+02,  2.43398438e+01,-9.02617188e+01],[-5.16093750e+01,  1.17328125e+02,  4.26406250e+01,-1.08359375e+02],[-4.12460938e+01,  1.27199219e+02,  1.49414062e+01,-1.00894531e+02],[-4.24531250e+01,  1.05484375e+02,  2.82968750e+01,-9.13281250e+01],[-4.26250000e+01,  1.08125000e+02,  2.73750000e+01,-9.28750000e+01],[-3.93203125e+01,  1.05664063e+02,  2.13046875e+01,-8.76484375e+01],[-5.14375000e+01,  1.14687500e+02,  4.35625000e+01,-1.06812500e+02],[-1.96875000e+01,  9.06562500e+01, -1.45000000e+01,-5.64687500e+01],[-2.50000000e+01,  9.50000000e+01, -5.00000000e+00,-6.50000000e+01],[-4.02343750e+01,  7.61093750e+01,  4.31406250e+01,-7.90156250e+01],[-2.71093750e+01,  1.01750000e+02, -4.57812500e+00,-7.00625000e+01],[-1.35351563e+01,  5.83476563e+01, -7.31640625e+00,-3.74960938e+01],[-1.68046875e+01,  5.49609375e+01,  2.78906250e+00,-4.09453125e+01],[-4.06523438e+01,  8.67304688e+01,  3.85664063e+01,-8.46445312e+01],[-6.35859375e+01,  1.36289062e+02,  5.90390625e+01,-1.31742188e+02],[-5.79492188e+01,  1.12371094e+02,  5.87695313e+01,-1.13191406e+02],[-6.15625000e+01,  1.54062500e+02,  4.79375000e+01,-1.40437500e+02],[-6.53125000e+00,  5.88125000e+01, -2.57187500e+01,-2.65625000e+01],[-1.60000000e+01,  7.70000000e+01, -1.40000000e+01,-4.70000000e+01],[-5.18359375e+00,  6.27734375e+00,  7.91015625e+00,-9.00390625e+00],[-6.25000000e+00,  1.32500000e+01,  5.75000000e+00,-1.27500000e+01],[-6.25000000e+00,  1.32500000e+01,  5.75000000e+00,-1.27500000e+01],[-6.25000000e+00,  1.32500000e+01,  5.75000000e+00,-1.27500000e+01],[-6.25000000e+00,  1.32500000e+01,  5.75000000e+00,-1.27500000e+01],[-6.66015625e+00,  1.36601562e+01,  6.43359375e+00,-1.34335937e+01],[-2.50000000e+00,  9.50000000e+00, -5.00000000e-01,-6.50000000e+00],[-2.55859375e+00,  9.55859375e+00, -4.02343750e-01,-6.59765625e+00],[-2.50000000e+00,  9.50000000e+00, -5.00000000e-01,-6.50000000e+00],[-2.50000000e+00,  9.50000000e+00, -5.00000000e-01,-6.50000000e+00],[-2.50000000e+00,  9.50000000e+00, -5.00000000e-01,-6.50000000e+00],[ 5.58593750e-01, -3.33203125e+00,  5.89843750e-01,2.18359375e+00],[-1.90937500e+01,  7.94921875e+01, -6.65625000e+00,-5.37421875e+01],[-2.40898437e+01,  9.73007812e+01, -6.87109375e+00,-6.63398437e+01],[-7.71875000e+00,  7.25781250e+00,  1.28437500e+01,-1.23828125e+01],[-1.02500000e+01,  2.12500000e+01,  9.75000000e+00,-2.07500000e+01],[-1.02500000e+01,  2.12500000e+01,  9.75000000e+00,-2.07500000e+01],[-1.02500000e+01,  2.12500000e+01,  9.75000000e+00,-2.07500000e+01],[-1.02500000e+01,  2.12500000e+01,  9.75000000e+00,-2.07500000e+01],[-1.02500000e+01,  2.12500000e+01,  9.75000000e+00,-2.07500000e+01],[-1.02500000e+01,  2.12500000e+01,  9.75000000e+00,-2.07500000e+01],[-1.02500000e+01,  2.12500000e+01,  9.75000000e+00,-2.07500000e+01],[-1.02500000e+01,  2.12500000e+01,  9.75000000e+00,-2.07500000e+01],[-1.15585937e+01,  2.39570312e+01,  1.10351562e+01,-2.34335937e+01],[-9.25000000e+00,  1.92500000e+01,  8.75000000e+00,-1.87500000e+01],[-1.22808594e+02,  2.52636719e+02,  1.20972656e+02,-2.50800781e+02],[-4.50000000e+00,  1.65000000e+01, -5.00000000e-01,-1.15000000e+01],[-5.16406250e+00,  1.78125000e+01,  2.10937500e-01,-1.28593750e+01],[-3.50000000e+00,  1.45000000e+01, -1.50000000e+00,-9.50000000e+00],[-3.50000000e+00,  1.45000000e+01, -1.50000000e+00,-9.50000000e+00],[-3.50000000e+00,  1.45000000e+01, -1.50000000e+00,-9.50000000e+00],[-3.50000000e+00,  1.45000000e+01, -1.50000000e+00,-9.50000000e+00],[-3.50000000e+00,  1.45000000e+01, -1.50000000e+00,-9.50000000e+00],[-3.50000000e+00,  1.45000000e+01, -1.50000000e+00,-9.50000000e+00],[-3.50000000e+00,  1.45000000e+01, -1.50000000e+00,-9.50000000e+00],[-3.50000000e+00,  1.45000000e+01, -1.50000000e+00,-9.50000000e+00],[-3.50000000e+00,  1.45000000e+01, -1.50000000e+00,-9.50000000e+00],[-3.50000000e+00,  1.45000000e+01, -1.50000000e+00,-9.50000000e+00],[-3.50000000e+00,  1.45000000e+01, -1.50000000e+00,-9.50000000e+00]]), 'da': array([[-8.37500000e+00, -1.58750000e+01,  1.00000000e+00,1.72500000e+01],[ 1.53437500e+01,  2.84687500e+01, -1.06250000e+00,-2.95000000e+01],[ 1.20625000e+01,  2.33125000e+01, -2.00000000e+00,-2.63750000e+01],[ 1.94375000e+01,  3.81875000e+01, -4.00000000e+00,-4.46250000e+01],[-2.03125000e+00, -3.90625000e+00,  3.12500000e-01,4.37500000e+00],[-2.32500000e+01, -4.57500000e+01,  4.87500000e+00,5.36250000e+01],[-6.21875000e+00, -1.18437500e+01,  8.12500000e-01,1.30000000e+01],[ 1.20625000e+01,  2.33125000e+01, -2.00000000e+00,-2.63750000e+01],[ 1.94375000e+01,  3.81875000e+01, -4.00000000e+00,-4.46250000e+01],[-2.03125000e+00, -3.90625000e+00,  3.12500000e-01,4.37500000e+00],[-2.71875000e+01, -5.34375000e+01,  5.62500000e+00,6.25000000e+01],[ 9.35937500e+00,  6.01562500e+00,  1.10156250e+01,1.81093750e+01],[ 6.03125000e+01,  5.58750000e+01, -3.17187500e+01,-6.57187500e+01],[ 9.54687500e+00,  1.09531250e+01,  2.51562500e+00,-2.51562500e+00],[ 7.53644380e-15, -1.25607397e-14, -2.76336273e-14,-4.77308107e-14],[-1.50728876e-14, -3.26579231e-14, -4.52186628e-14,-4.77308107e-14],[-4.35000000e+01, -6.81875000e+01, -9.06250000e-01,5.73437500e+01],[ 1.67500000e+01,  1.64375000e+01,  5.90625000e+00,2.15625000e+00],[ 5.02429587e-15,  1.75850355e-14,  1.50728876e-14,2.26093314e-14],[ 3.01457752e-14, -4.39625888e-14,  6.02915504e-14,2.38654054e-14],[-3.46875000e+00, -6.59375000e+00,  5.00000000e-01,7.31250000e+00],[-1.08656250e+02, -1.41781250e+02, -1.86250000e+01,7.33125000e+01],[-8.43750000e+01, -9.56250000e+01, -2.21250000e+01,2.21250000e+01],[-2.51214793e-15,  6.78279942e-14, -5.77794025e-14,-4.77308107e-14],[ 4.01943669e-14,  6.28036983e-14,  2.00971835e-14,1.75850355e-14],[ 1.02812500e+01,  1.96562500e+01, -1.62500000e+00,-2.20625000e+01],[ 3.85625000e+01,  5.70000000e+01,  2.40625000e+00,-4.32187500e+01],[-2.20937500e+01, -2.49062500e+01, -5.78125000e+00,5.78125000e+00],[ 2.51214793e-15,  1.75850355e-14,  1.75850355e-14,4.27065149e-14],[ 1.00485917e-14,  1.75850355e-14,  1.00485917e-14,-1.75850355e-14],[ 2.09375000e+01,  3.96875000e+01, -2.87500000e+00,-4.37500000e+01],[ 3.45625000e+01,  6.58125000e+01, -5.12500000e+00,-7.32500000e+01],[ 1.39062500e+01,  1.22812500e+01,  1.81562500e+01,2.31562500e+01],[ 1.07343750e+02,  9.59687500e+01,  1.37093750e+02,1.72093750e+02],[ 4.05937500e+01,  2.92187500e+01,  7.03437500e+01,1.05343750e+02],[-1.46515625e+02, -1.53203125e+02, -1.65046875e+02,-1.68484375e+02],[ 2.15625000e+01,  2.43125000e+01,  2.19062500e+01,1.84687500e+01],[-4.32187500e+01, -6.30000000e+01, -3.28125000e+01,2.03125000e+00],[ 7.85312500e+01,  8.77187500e+01, -4.61562500e+01,-1.20093750e+02],[ 1.15468750e+01,  1.29531250e+01,  3.01562500e+00,-3.01562500e+00],[-1.31015625e+02, -1.30453125e+02, -8.95781250e+01,-7.49531250e+01],[-2.83593750e+01,  9.07812500e+00, -2.37968750e+01,-8.29218750e+01],[ 9.89062500e+00,  9.57812500e+00,  1.52656250e+01,1.92656250e+01],[ 7.92343750e+01,  7.70468750e+01,  1.16859375e+02,1.44859375e+02],[-2.53750000e+01, -2.53750000e+01, -2.53750000e+01,-2.53750000e+01],[-3.62500000e+01, -3.62500000e+01, -3.62500000e+01,-3.62500000e+01],[-1.72710170e-15, -4.86728662e-15, -2.04112020e-15,-3.92523115e-15],[ 3.62500000e+01,  3.62500000e+01,  3.62500000e+01,3.62500000e+01],[ 2.53750000e+01,  2.53750000e+01,  2.53750000e+01,2.53750000e+01],[ 1.09375000e-01, -2.07812500e+00,  2.29687500e+00,6.67187500e+00],[-3.26093750e+01, -3.29218750e+01, -3.22968750e+01,-3.16718750e+01],[ 1.00485917e-14, -1.00485917e-14,  1.00485917e-14,-1.00485917e-14],[ 3.62500000e+01,  3.62500000e+01,  3.62500000e+01,3.62500000e+01],[ 2.53750000e+01,  2.53750000e+01,  2.53750000e+01,2.53750000e+01],[-2.53750000e+01, -2.53750000e+01, -2.53750000e+01,-2.53750000e+01],[-7.41484375e+01, -7.11953125e+01, -1.94453125e+01,1.07890625e+01],[ 1.20937500e+01,  1.20000000e+01, -1.75000000e+00,-1.03437500e+01],[-3.21953125e+01, -3.43046875e+01,  3.76953125e+01,8.48046875e+01],[ 4.28750000e+01,  4.28750000e+01,  4.28750000e+01,4.28750000e+01],[-4.28750000e+01, -4.28750000e+01, -4.28750000e+01,-4.28750000e+01],[-6.12500000e+01, -6.12500000e+01, -6.12500000e+01,-6.12500000e+01],[ 5.02429587e-14,  1.50728876e-14,  7.03401421e-14,2.51214793e-14],[ 6.12500000e+01,  6.12500000e+01,  6.12500000e+01,6.12500000e+01],[ 4.28750000e+01,  4.28750000e+01,  4.28750000e+01,4.28750000e+01],[-4.28750000e+01, -4.28750000e+01, -4.28750000e+01,-4.28750000e+01],[-6.12500000e+01, -6.12500000e+01, -6.12500000e+01,-6.12500000e+01],[ 3.76822190e-15,  7.53644380e-15, -2.38654054e-14,2.51214793e-15],[ 6.23984375e+01,  5.73203125e+01,  5.91953125e+01,6.40859375e+01],[ 4.59140625e+01,  1.43671875e+01,  2.54921875e+01,5.57265625e+01],[ 1.97109375e+01, -3.07734375e+01, -4.27343750e+00,4.88359375e+01],[-5.30234375e+01, -5.96640625e+01, -5.61640625e+01,-4.91484375e+01],[-3.26579231e-14,  1.00485917e-14, -7.53644380e-15,-3.01457752e-14],[ 6.12500000e+01,  6.12500000e+01,  6.12500000e+01,6.12500000e+01],[ 4.28750000e+01,  4.28750000e+01,  4.28750000e+01,4.28750000e+01],[-4.28750000e+01, -4.28750000e+01, -4.28750000e+01,-4.28750000e+01],[-6.12500000e+01, -6.12500000e+01, -6.12500000e+01,-6.12500000e+01],[ 7.53644380e-15, -4.77308107e-14,  5.27551066e-14,2.76336273e-14],[ 6.12500000e+01,  6.12500000e+01,  6.12500000e+01,6.12500000e+01],[ 4.28750000e+01,  4.28750000e+01,  4.28750000e+01,4.28750000e+01],[-4.28750000e+01, -4.28750000e+01, -4.28750000e+01,-4.28750000e+01],[-1.73750000e+01, -1.73750000e+01, -1.73750000e+01,-1.73750000e+01],[-1.00000000e+01, -1.00000000e+01, -1.00000000e+01,-1.00000000e+01],[ 5.63750000e+01,  5.63750000e+01,  5.63750000e+01,5.63750000e+01]]), 'dd': array([[ 2.65625000e+00, -2.03125000e+00, -4.84375000e+00,4.21875000e+00],[-4.64843750e+00,  3.55468750e+00,  8.47656250e+00,-7.38281250e+00],[-3.98437500e+00,  3.04687500e+00,  7.26562500e+00,-6.32812500e+00],[-6.64062500e+00,  5.07812500e+00,  1.21093750e+01,-1.05468750e+01],[ 6.64062500e-01, -5.07812500e-01, -1.21093750e+00,1.05468750e+00],[ 7.96875000e+00, -6.09375000e+00, -1.45312500e+01,1.26562500e+01],[ 1.99218750e+00, -1.52343750e+00, -3.63281250e+00,3.16406250e+00],[-3.98437500e+00,  3.04687500e+00,  7.26562500e+00,-6.32812500e+00],[-6.64062500e+00,  5.07812500e+00,  1.21093750e+01,-1.05468750e+01],[ 6.64062500e-01, -5.07812500e-01, -1.21093750e+00,1.05468750e+00],[ 9.29687500e+00, -7.10937500e+00, -1.69531250e+01,1.47656250e+01],[ 1.53125000e+00, -2.31250000e+00, -2.46875000e+00,3.25000000e+00],[ 1.16171875e+01, -4.68359375e+01, -2.00781250e+00,3.72265625e+01],[ 7.81250000e-02, -2.17187500e+00,  8.28125000e-01,1.26562500e+00],[ 1.00485917e-14, -2.51214793e-14,  0.00000000e+00,1.50728876e-14],[ 1.00485917e-14, -2.51214793e-14, -5.02429587e-15,2.00971835e-14],[ 7.49218750e+00, -1.08593750e+00, -1.58828125e+01,9.47656250e+00],[ 1.32031250e+00, -5.47656250e+00, -3.04687500e-01,4.46093750e+00],[-5.02429587e-15,  1.00485917e-14,  0.00000000e+00,-5.02429587e-15],[ 1.75850355e-14, -1.00485917e-14,  1.00485917e-14,-1.75850355e-14],[ 1.10156250e+00, -8.20312500e-01, -2.02343750e+00,1.74218750e+00],[ 6.58593750e+00,  1.41328125e+01, -2.12890625e+01,5.70312500e-01],[-1.12500000e+00,  1.98750000e+01, -7.12500000e+00,-1.16250000e+01],[-1.50728876e-14,  0.00000000e+00, -5.02429587e-15,2.00971835e-14],[-5.02429587e-15,  0.00000000e+00,  0.00000000e+00,5.02429587e-15],[-3.30468750e+00,  2.46093750e+00,  6.07031250e+00,-5.22656250e+00],[-5.16406250e+00, -1.17968750e+00,  1.19609375e+01,-5.61718750e+00],[-3.43750000e-01,  5.28125000e+00, -1.84375000e+00,-3.09375000e+00],[-5.02429587e-15,  1.00485917e-14, -5.02429587e-15,0.00000000e+00],[-5.02429587e-15,  1.00485917e-14,  1.00485917e-14,-1.50728876e-14],[-6.60937500e+00,  4.92187500e+00,  1.21406250e+01,-1.04531250e+01],[-1.10156250e+01,  8.20312500e+00,  2.02343750e+01,-1.74218750e+01],[ 3.90625000e-01,  4.84375000e-01, -1.23437500e+00,3.59375000e-01],[ 2.73437500e+00,  3.39062500e+00, -8.64062500e+00,2.51562500e+00],[ 2.73437500e+00,  3.39062500e+00, -8.64062500e+00,2.51562500e+00],[ 4.60937500e+00, -1.30156250e+01, -2.07812500e+00,1.04843750e+01],[-1.28906250e+00,  2.32031250e+00,  1.46093750e+00,-2.49218750e+00],[ 8.33593750e+00, -1.16796875e+01, -1.21015625e+01,1.54453125e+01],[ 8.09375000e+00, -4.96562500e+01,  8.09375000e+00,3.34687500e+01],[ 2.03125000e-01, -2.79687500e+00,  9.53125000e-01,1.64062500e+00],[-4.80468750e+00,  2.05546875e+01,  1.00781250e+00,-1.67578125e+01],[-1.86328125e+01,  3.43828125e+01,  2.40546875e+01,-3.98046875e+01],[-3.04687500e-01,  1.99218750e+00, -6.17187500e-01,-1.07031250e+00],[-2.13281250e+00,  1.39453125e+01, -4.32031250e+00,-7.49218750e+00],[ 8.88178420e-16,  8.88178420e-16,  1.77635684e-15,-3.55271368e-15],[-1.77635684e-15,  0.00000000e+00,  0.00000000e+00,1.77635684e-15],[ 9.42055475e-16, -1.25607397e-15,  3.14018492e-16,-4.93038066e-32],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-2.66453526e-15,  6.21724894e-15,  8.88178420e-16,-4.44089210e-15],[ 8.20312500e-01, -8.20312500e-01, -1.36718750e+00,1.36718750e+00],[ 1.17187500e-01, -1.17187500e-01, -1.95312500e-01,1.95312500e-01],[ 5.02429587e-15, -5.02429587e-15,  5.02429587e-15,-5.02429587e-15],[ 0.00000000e+00, -3.55271368e-15,  3.55271368e-15,0.00000000e+00],[ 0.00000000e+00,  4.44089210e-15,  2.66453526e-15,-7.10542736e-15],[ 2.66453526e-15, -7.10542736e-15, -1.77635684e-15,6.21724894e-15],[-6.11718750e+00,  2.56640625e+01, -2.17968750e+00,-1.73671875e+01],[ 1.24218750e+00, -5.96093750e+00,  9.29687500e-01,3.78906250e+00],[-5.06250000e+00,  2.79843750e+01, -6.18750000e+00,-1.67343750e+01],[-5.32907052e-15,  1.06581410e-14,  1.06581410e-14,-1.59872116e-14],[ 5.32907052e-15, -2.13162821e-14, -5.32907052e-15,2.13162821e-14],[-5.32907052e-15,  0.00000000e+00, -2.30926389e-14,2.84217094e-14],[ 5.02429587e-15,  5.02429587e-15,  1.50728876e-14,-2.51214793e-14],[ 1.42108547e-14, -1.95399252e-14, -5.32907052e-15,1.06581410e-14],[ 1.95399252e-14, -3.01980663e-14, -1.06581410e-14,2.13162821e-14],[-5.32907052e-15,  1.06581410e-14,  5.32907052e-15,-1.06581410e-14],[ 1.24344979e-14, -3.01980663e-14, -1.06581410e-14,2.84217094e-14],[ 5.02429587e-15, -2.00971835e-14, -1.25607397e-14,2.76336273e-14],[ 2.61718750e+00, -5.41406250e+00, -2.57031250e+00,5.36718750e+00],[ 1.63203125e+01, -3.38984375e+01, -1.59921875e+01,3.35703125e+01],[ 2.52968750e+01, -5.03750000e+01, -2.59531250e+01,5.10312500e+01],[ 3.32812500e+00, -6.62500000e+00, -3.42187500e+00,6.71875000e+00],[-2.00971835e-14,  4.01943669e-14,  1.50728876e-14,-3.51700711e-14],[-1.95399252e-14,  3.01980663e-14,  0.00000000e+00,-1.06581410e-14],[-2.13162821e-14,  2.13162821e-14, -1.06581410e-14,1.06581410e-14],[-8.88178420e-15,  1.42108547e-14,  1.06581410e-14,-1.59872116e-14],[ 8.88178420e-15, -2.30926389e-14,  5.32907052e-15,8.88178420e-15],[ 1.00485917e-14,  5.02429587e-15,  1.00485917e-14,-2.51214793e-14],[ 8.88178420e-15, -1.77635684e-14, -2.48689958e-14,3.37507799e-14],[-1.59872116e-14,  3.19744231e-14,  5.32907052e-15,-2.13162821e-14],[ 0.00000000e+00,  3.55271368e-15,  2.48689958e-14,-2.84217094e-14],[-2.57571742e-14,  4.61852778e-14,  9.76996262e-15,-3.01980663e-14],[ 1.50990331e-14, -3.01980663e-14, -9.76996262e-15,2.48689958e-14],[ 1.42108547e-14, -1.77635684e-14, -8.88178420e-15,1.24344979e-14]])}, {'ad': array([[ 1.23750e+01, -5.31250e+00,  1.23750e+01, -1.94375e+01],[ 4.87500e+00,  1.09375e+01,  4.87500e+00, -2.06875e+01],[ 1.05000e+01, -1.25000e+00,  1.05000e+01, -1.97500e+01],[ 8.62500e+00,  2.81250e+00,  8.62500e+00, -2.00625e+01],[ 1.80000e+01, -1.75000e+01,  1.80000e+01, -1.85000e+01],[ 4.87500e+00,  1.09375e+01,  4.87500e+00, -2.06875e+01],[ 1.05000e+01, -1.25000e+00,  1.05000e+01, -1.97500e+01],[ 1.05000e+01, -1.25000e+00,  1.05000e+01, -1.97500e+01],[ 8.62500e+00,  2.81250e+00,  8.62500e+00, -2.00625e+01],[ 1.80000e+01, -1.75000e+01,  1.80000e+01, -1.85000e+01],[ 4.87500e+00,  1.09375e+01,  4.87500e+00, -2.06875e+01],[ 1.05000e+01, -1.25000e+00,  1.05000e+01, -1.97500e+01],[ 1.05000e+01, -1.25000e+00,  1.05000e+01, -1.97500e+01],[ 8.62500e+00,  2.81250e+00,  8.62500e+00, -2.00625e+01],[ 1.80000e+01, -1.75000e+01,  1.80000e+01, -1.85000e+01],[ 4.87500e+00,  1.09375e+01,  4.87500e+00, -2.06875e+01],[ 1.05000e+01, -1.25000e+00,  1.05000e+01, -1.97500e+01],[ 1.05000e+01, -1.25000e+00,  1.05000e+01, -1.97500e+01],[ 8.62500e+00,  2.81250e+00,  8.62500e+00, -2.00625e+01],[ 1.80000e+01, -1.75000e+01,  1.80000e+01, -1.85000e+01],[ 1.12500e+00,  1.90625e+01,  1.12500e+00, -2.13125e+01],[ 3.00000e+00,  1.50000e+01,  3.00000e+00, -2.10000e+01],[ 1.62500e+00,  1.56875e+01,  1.25000e-01, -1.74375e+01],[ 1.18750e+01,  3.64375e+01,  2.08750e+01, -6.91875e+01],[ 1.90000e+01, -9.62500e+00,  1.75000e+01, -2.68750e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 2.68750e+01, -2.26875e+01,  2.68750e+01, -3.10625e+01],[ 1.13750e+01,  1.98125e+01,  1.43750e+01, -4.55625e+01],[ 1.45000e+01,  1.30000e+01,  1.75000e+01, -4.50000e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.43750e+01,  4.56250e+00,  1.43750e+01, -3.33125e+01],[ 2.83750e+01, -3.05625e+01,  2.68750e+01, -2.46875e+01],[ 8.50000e+00,  4.50000e+01,  1.75000e+01, -7.10000e+01],[ 1.90000e+01, -1.01250e+01,  1.75000e+01, -2.63750e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 2.68750e+01, -2.26875e+01,  2.68750e+01, -3.10625e+01],[ 1.13750e+01,  2.08125e+01,  1.43750e+01, -4.65625e+01],[ 1.45000e+01,  1.40000e+01,  1.75000e+01, -4.60000e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.75000e+01, -2.25000e+00,  1.75000e+01, -3.27500e+01],[ 1.43750e+01,  4.56250e+00,  1.43750e+01, -3.33125e+01],[ 3.00000e+01, -2.95000e+01,  3.00000e+01, -3.05000e+01],[ 1.87500e+00,  3.18125e+01,  1.87500e+00, -3.55625e+01],[ 5.00000e+00,  2.50000e+01,  5.00000e+00, -3.50000e+01],[ 5.00000e+00,  2.50000e+01,  5.00000e+00, -3.50000e+01],[ 3.37500e+00,  3.00000e+01,  3.37500e+00, -3.67500e+01],[ 1.63750e+01, -1.00000e+01,  1.63750e+01, -2.27500e+01],[ 1.80000e+01, -1.50000e+01,  1.80000e+01, -2.10000e+01],[ 1.63750e+01, -1.00000e+01,  1.63750e+01, -2.27500e+01],[ 3.37500e+00,  3.00000e+01,  3.37500e+00, -3.67500e+01],[ 5.56250e+00,  2.59375e+01,  5.56250e+00, -3.70625e+01],[ 1.06250e+00,  1.84375e+01,  1.06250e+00, -2.05625e+01],[ 5.00000e-01,  1.75000e+01,  5.00000e-01, -1.85000e+01],[ 2.68750e+00,  1.31250e+01,  2.68750e+00, -1.85000e+01],[ 2.01875e+01, -2.18750e+01,  2.01875e+01, -1.85000e+01],[ 1.80000e+01, -2.43750e+01,  1.65000e+01, -1.01250e+01],[ 1.95000e+01,  2.22500e+01,  2.85000e+01, -7.02500e+01],[ 3.15000e+01, -3.78750e+01,  3.00000e+01, -2.36250e+01],[ 3.00000e+01, -2.95000e+01,  3.00000e+01, -3.05000e+01],[ 3.00000e+01, -2.95000e+01,  3.00000e+01, -3.05000e+01],[ 2.95625e+01, -3.27500e+01,  2.80625e+01, -2.48750e+01],[ 2.89375e+01, -2.62500e+00,  3.79375e+01, -6.42500e+01],[-3.62500e+00,  3.36250e+01, -5.12500e+00, -2.48750e+01],[ 5.00000e-01,  2.95000e+01,  5.00000e-01, -3.05000e+01],[ 5.00000e-01,  2.95000e+01,  5.00000e-01, -3.05000e+01],[ 1.87500e-01,  3.35000e+01,  1.87500e-01, -3.38750e+01],[ 2.68750e+00,  1.50000e+00,  2.68750e+00, -6.87500e+00],[ 3.00000e+00, -2.50000e+00,  3.00000e+00, -3.50000e+00],[ 3.00000e+00, -2.50000e+00,  3.00000e+00, -3.50000e+00],[ 3.00000e+00, -2.50000e+00,  3.00000e+00, -3.50000e+00],[ 3.00000e+00, -2.50000e+00,  3.00000e+00, -3.50000e+00],[ 3.00000e+00, -2.50000e+00,  3.00000e+00, -3.50000e+00],[ 3.00000e+00, -2.50000e+00,  3.00000e+00, -3.50000e+00],[ 3.00000e+00, -2.50000e+00,  3.00000e+00, -3.50000e+00],[ 3.00000e+00, -2.50000e+00,  3.00000e+00, -3.50000e+00],[ 3.00000e+00, -2.50000e+00,  3.00000e+00, -3.50000e+00],[ 3.00000e+00, -2.50000e+00,  3.00000e+00, -3.50000e+00],[ 3.00000e+00, -2.50000e+00,  3.00000e+00, -3.50000e+00],[ 2.68750e+00, -1.87500e+00,  2.68750e+00, -3.50000e+00],[ 1.87500e-01,  3.12500e+00,  1.87500e-01, -3.50000e+00],[ 5.00000e-01,  2.50000e+00,  5.00000e-01, -3.50000e+00],[ 5.00000e-01,  2.50000e+00,  5.00000e-01, -3.50000e+00],[ 5.00000e-01,  2.50000e+00,  5.00000e-01, -3.50000e+00],[ 5.00000e-01,  2.50000e+00,  5.00000e-01, -3.50000e+00],[ 5.00000e-01,  2.50000e+00,  5.00000e-01, -3.50000e+00],[ 5.00000e-01,  2.50000e+00,  5.00000e-01, -3.50000e+00],[ 5.00000e-01,  2.50000e+00,  5.00000e-01, -3.50000e+00],[ 5.00000e-01,  2.50000e+00,  5.00000e-01, -3.50000e+00],[ 5.00000e-01,  2.50000e+00,  5.00000e-01, -3.50000e+00],[ 5.00000e-01,  2.50000e+00,  5.00000e-01, -3.50000e+00],[ 1.87500e-01, -1.25000e+00, -6.25000e-02,  1.12500e+00],[ 2.68750e+00,  2.87500e+01,  4.43750e+00, -3.58750e+01],[ 3.00000e+00,  3.65000e+01,  5.00000e+00, -4.45000e+01],[ 3.25000e+00,  3.48750e+01,  5.00000e+00, -4.31250e+01],[ 5.25000e+00, -1.01250e+01,  5.00000e+00, -1.25000e-01],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[ 5.00000e+00, -4.50000e+00,  5.00000e+00, -5.50000e+00],[-6.25000e-02,  0.00000e+00, -3.12500e-01,  3.75000e-01],[ 4.04375e+01, -3.60000e+01,  4.21875e+01, -4.66250e+01],[ 4.95000e+01, -4.45000e+01,  5.15000e+01, -5.65000e+01],[ 4.68750e+01, -4.18750e+01,  4.86250e+01, -5.36250e+01],[-6.12500e+00,  1.11250e+01, -6.37500e+00,  1.37500e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00],[ 5.00000e-01,  4.50000e+00,  5.00000e-01, -5.50000e+00]]), 'da': array([[ 1.53125000e+01,  4.68750000e+00,  2.18750000e+00,-1.71875000e+01],[-3.01250000e+01, -8.87500000e+00, -3.87500000e+00,3.48750000e+01],[-3.01250000e+01, -8.87500000e+00, -3.87500000e+00,3.48750000e+01],[-1.48125000e+01, -4.18750000e+00, -1.68750000e+00,1.76875000e+01],[ 1.63125000e+01,  5.68750000e+00,  3.18750000e+00,-1.61875000e+01],[-3.01250000e+01, -8.87500000e+00, -3.87500000e+00,3.48750000e+01],[-3.01250000e+01, -8.87500000e+00, -3.87500000e+00,3.48750000e+01],[-3.01250000e+01, -8.87500000e+00, -3.87500000e+00,3.48750000e+01],[-1.48125000e+01, -4.18750000e+00, -1.68750000e+00,1.76875000e+01],[ 1.63125000e+01,  5.68750000e+00,  3.18750000e+00,-1.61875000e+01],[-3.01250000e+01, -8.87500000e+00, -3.87500000e+00,3.48750000e+01],[-3.01250000e+01, -8.87500000e+00, -3.87500000e+00,3.48750000e+01],[-3.01250000e+01, -8.87500000e+00, -3.87500000e+00,3.48750000e+01],[-1.48125000e+01, -4.18750000e+00, -1.68750000e+00,1.76875000e+01],[ 1.63125000e+01,  5.68750000e+00,  3.18750000e+00,-1.61875000e+01],[-3.01250000e+01, -8.87500000e+00, -3.87500000e+00,3.48750000e+01],[-3.01250000e+01, -8.87500000e+00, -3.87500000e+00,3.48750000e+01],[-3.01250000e+01, -8.87500000e+00, -3.87500000e+00,3.48750000e+01],[-1.48125000e+01, -4.18750000e+00, -1.68750000e+00,1.76875000e+01],[ 1.63125000e+01,  5.68750000e+00,  3.18750000e+00,-1.61875000e+01],[ 3.14018492e-16, -2.51214793e-15, -3.14018492e-16,-5.02429587e-15],[-1.57009246e-15,  2.19812944e-15, -9.42055475e-16,3.14018492e-16],[-3.88125000e+01, -5.26875000e+01, -3.76875000e+01,-5.78125000e+01],[ 1.15000000e+01, -2.50000000e+01, -5.00000000e-01,-5.20000000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[-2.58125000e+01, -8.18750000e+00, -3.68750000e+00,2.86875000e+01],[ 3.16250000e+01,  7.03750000e+01,  8.37500000e+00,4.56250000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-2.53125000e+01, -7.68750000e+00, -3.18750000e+00,2.91875000e+01],[ 8.00000000e+00, -2.95000000e+01, -1.00000000e+00,-5.15000000e+01],[ 8.00000000e+00, -2.95000000e+01, -1.00000000e+00,-5.15000000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[ 5.11250000e+01,  1.58750000e+01,  6.87500000e+00,-5.78750000e+01],[-2.58125000e+01, -8.18750000e+00, -3.68750000e+00,2.86875000e+01],[ 3.86250000e+01,  7.93750000e+01,  9.37500000e+00,4.46250000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-5.11250000e+01, -1.58750000e+01, -6.87500000e+00,5.78750000e+01],[-2.53125000e+01, -7.68750000e+00, -3.18750000e+00,2.91875000e+01],[ 2.58125000e+01,  8.18750000e+00,  3.68750000e+00,-2.86875000e+01],[-2.51214793e-15, -1.38168136e-14,  1.25607397e-14,-6.28036983e-15],[-7.53644380e-15, -1.13046657e-14, -2.51214793e-15,-8.79251777e-15],[-5.02429587e-15,  2.51214793e-15, -1.50728876e-14,-2.51214793e-15],[-5.08750000e+01, -5.71250000e+01, -7.11250000e+01,-9.08750000e+01],[ 1.25607397e-15,  6.28036983e-16, -1.25607397e-15,-6.28036983e-16],[ 1.57009246e-16,  0.00000000e+00, -1.57009246e-16,-1.25607397e-15],[-1.38750000e+01, -2.01250000e+01, -3.41250000e+01,-5.38750000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-7.15000000e+01, -6.40000000e+01, -8.05000000e+01,-7.90000000e+01],[ 6.86915451e-16, -3.04205414e-16,  1.90373711e-15,3.04205414e-16],[ 9.22429319e-16, -4.71027738e-16,  1.58971861e-15,-2.19812944e-15],[-2.78125000e+01, -1.46875000e+01, -1.46875000e+01,7.18750000e+00],[-3.14018492e-16,  0.00000000e+00,  3.14018492e-16,2.51214793e-15],[ 8.37500000e+00, -6.87500000e+00,  4.66250000e+01,4.98750000e+01],[-4.66250000e+01, -4.98750000e+01, -8.37500000e+00,6.87500000e+00],[ 1.25607397e-14, -7.53644380e-15,  7.53644380e-15,7.53644380e-15],[-2.51214793e-15, -1.00485917e-14,  2.51214793e-15,1.00485917e-14],[-8.79251777e-15, -5.02429587e-15,  8.79251777e-15,-5.02429587e-15],[-5.43125000e+01, -6.66875000e+01, -3.51875000e+01,-4.18125000e+01],[ 1.50000000e+00, -3.30000000e+01, -1.50000000e+00,-4.50000000e+01],[ 3.76822190e-15, -1.25607397e-15,  6.28036983e-15,-8.79251777e-15],[ 6.28036983e-16,  3.14018492e-15,  4.39625888e-15,-8.16448079e-15],[-2.51214793e-15,  1.25607397e-15, -7.53644380e-15,-1.25607397e-15],[-4.16875000e+01, -3.68125000e+01, -6.38125000e+01,-7.36875000e+01],[-5.98597750e-16,  2.35513869e-16, -1.28551320e-15,7.85046229e-16],[-7.85046229e-16, -3.14018492e-16,  7.85046229e-16,1.57009246e-15],[ 1.45000000e+01,  1.45000000e+01,  1.45000000e+01,1.45000000e+01],[ 9.42055475e-16, -1.88411095e-15,  1.57009246e-15,6.28036983e-16],[-1.45000000e+01, -1.45000000e+01, -1.45000000e+01,-1.45000000e+01],[ 6.47663139e-16, -4.71027738e-16,  6.08410828e-16,-6.28036983e-16],[-2.04112020e-15,  1.09906472e-15, -1.72710170e-15,1.57009246e-16],[ 1.45000000e+01,  1.45000000e+01,  1.45000000e+01,1.45000000e+01],[-2.35513869e-16,  7.85046229e-16, -1.02056010e-15,-1.57009246e-16],[-1.45000000e+01, -1.45000000e+01, -1.45000000e+01,-1.45000000e+01],[-2.74766180e-16,  4.71027738e-16, -9.81307787e-16,-1.57009246e-16],[ 7.85046229e-16, -6.28036983e-16, -7.85046229e-16,1.88411095e-15],[ 1.18750000e+00, -6.87500000e-01, -6.87500000e-01,-3.81250000e+00],[-2.19812944e-15, -2.82616643e-15, -3.14018492e-16,3.14018492e-16],[-1.45000000e+01, -1.45000000e+01, -1.45000000e+01,-1.45000000e+01],[-1.25607397e-15,  2.19812944e-15, -1.25607397e-15,-2.19812944e-15],[ 1.88411095e-15,  1.25607397e-15, -1.88411095e-15,3.76822190e-15],[ 1.45000000e+01,  1.45000000e+01,  1.45000000e+01,1.45000000e+01],[ 1.88411095e-15,  1.25607397e-15, -1.88411095e-15,3.76822190e-15],[-1.45000000e+01, -1.45000000e+01, -1.45000000e+01,-1.45000000e+01],[ 1.88411095e-15,  1.25607397e-15, -1.88411095e-15,3.76822190e-15],[-6.28036983e-16, -5.02429587e-15,  6.28036983e-16,0.00000000e+00],[ 1.45000000e+01,  1.45000000e+01,  1.45000000e+01,1.45000000e+01],[-6.28036983e-16, -5.02429587e-15,  6.28036983e-16,0.00000000e+00],[-2.56875000e+01, -3.65625000e+01, -1.81250000e+00,2.06250000e+00],[ 9.42055475e-16, -7.85046229e-16,  2.82616643e-15,-4.23924964e-15],[-1.57009246e-15,  2.19812944e-15, -9.42055475e-16,3.14018492e-16],[-1.28750000e+01, -2.18750000e+01,  1.88750000e+01,2.98750000e+01],[ 6.28036983e-16,  5.02429587e-15, -6.28036983e-16,0.00000000e+00],[-2.45000000e+01, -2.45000000e+01, -2.45000000e+01,-2.45000000e+01],[-1.88411095e-15,  6.28036983e-16, -3.14018492e-15,4.39625888e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 2.45000000e+01,  2.45000000e+01,  2.45000000e+01,2.45000000e+01],[-6.28036983e-16, -5.02429587e-15,  6.28036983e-16,0.00000000e+00],[-2.45000000e+01, -2.45000000e+01, -2.45000000e+01,-2.45000000e+01],[-1.88411095e-15,  6.28036983e-16, -3.14018492e-15,4.39625888e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 2.45000000e+01,  2.45000000e+01,  2.45000000e+01,2.45000000e+01],[-1.25607397e-15, -5.02429587e-15,  1.25607397e-15,5.02429587e-15],[-2.45000000e+01, -2.45000000e+01, -2.45000000e+01,-2.45000000e+01],[ 3.14018492e-15, -4.39625888e-15,  1.88411095e-15,-6.28036983e-16],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 2.45000000e+01,  2.45000000e+01,  2.45000000e+01,2.45000000e+01],[ 6.28036983e-16,  0.00000000e+00, -6.28036983e-16,-5.02429587e-15],[-2.45000000e+01, -2.45000000e+01, -2.45000000e+01,-2.45000000e+01],[-3.14018492e-15,  4.39625888e-15, -1.88411095e-15,6.28036983e-16],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 2.45000000e+01,  2.45000000e+01,  2.45000000e+01,2.45000000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-3.18750000e+00, -4.50625000e+01, -3.12500000e-01,-4.14375000e+01],[-3.14018492e-16, -2.51214793e-15,  3.14018492e-16,0.00000000e+00],[ 2.51214793e-15, -1.25607397e-15,  7.53644380e-15,1.25607397e-15],[ 2.47500000e+01, -2.85000000e+01,  2.42500000e+01,-3.05000000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-2.45000000e+01, -2.45000000e+01, -2.45000000e+01,-2.45000000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-3.76822190e-15,  1.25607397e-15, -6.28036983e-15,8.79251777e-15],[ 2.45000000e+01,  2.45000000e+01,  2.45000000e+01,2.45000000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-2.45000000e+01, -2.45000000e+01, -2.45000000e+01,-2.45000000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 2.45000000e+01,  2.45000000e+01,  2.45000000e+01,2.45000000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-2.45000000e+01, -2.45000000e+01, -2.45000000e+01,-2.45000000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 1.25607397e-15, -1.25607397e-15,  8.79251777e-15,1.13046657e-14],[ 2.45000000e+01,  2.45000000e+01,  2.45000000e+01,2.45000000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-2.45000000e+01, -2.45000000e+01, -2.45000000e+01,-2.45000000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 1.25607397e-15,  0.00000000e+00, -1.25607397e-15,-1.00485917e-14],[ 2.45000000e+01,  2.45000000e+01,  2.45000000e+01,2.45000000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-5.00000000e+00, -5.00000000e+00, -5.00000000e+00,-5.00000000e+00],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00]]), 'dd': array([[ 3.75000000e+00, -8.12500000e+00,  3.75000000e+00,6.25000000e-01],[-7.50000000e+00,  1.62500000e+01, -7.50000000e+00,-1.25000000e+00],[-7.50000000e+00,  1.62500000e+01, -7.50000000e+00,-1.25000000e+00],[-3.75000000e+00,  8.12500000e+00, -3.75000000e+00,-6.25000000e-01],[ 3.75000000e+00, -8.12500000e+00,  3.75000000e+00,6.25000000e-01],[-7.50000000e+00,  1.62500000e+01, -7.50000000e+00,-1.25000000e+00],[-7.50000000e+00,  1.62500000e+01, -7.50000000e+00,-1.25000000e+00],[-7.50000000e+00,  1.62500000e+01, -7.50000000e+00,-1.25000000e+00],[-3.75000000e+00,  8.12500000e+00, -3.75000000e+00,-6.25000000e-01],[ 3.75000000e+00, -8.12500000e+00,  3.75000000e+00,6.25000000e-01],[-7.50000000e+00,  1.62500000e+01, -7.50000000e+00,-1.25000000e+00],[-7.50000000e+00,  1.62500000e+01, -7.50000000e+00,-1.25000000e+00],[-7.50000000e+00,  1.62500000e+01, -7.50000000e+00,-1.25000000e+00],[-3.75000000e+00,  8.12500000e+00, -3.75000000e+00,-6.25000000e-01],[ 3.75000000e+00, -8.12500000e+00,  3.75000000e+00,6.25000000e-01],[-7.50000000e+00,  1.62500000e+01, -7.50000000e+00,-1.25000000e+00],[-7.50000000e+00,  1.62500000e+01, -7.50000000e+00,-1.25000000e+00],[-7.50000000e+00,  1.62500000e+01, -7.50000000e+00,-1.25000000e+00],[-3.75000000e+00,  8.12500000e+00, -3.75000000e+00,-6.25000000e-01],[ 3.75000000e+00, -8.12500000e+00,  3.75000000e+00,6.25000000e-01],[ 6.28036983e-16, -6.28036983e-16,  1.25607397e-15,-1.25607397e-15],[-1.25607397e-15,  1.88411095e-15,  0.00000000e+00,-6.28036983e-16],[ 2.75000000e+00, -1.37500000e+00,  5.75000000e+00,-7.12500000e+00],[ 9.50000000e+00, -1.25000000e+01,  1.25000000e+01,-9.50000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[-6.25000000e+00,  1.36250000e+01, -6.25000000e+00,-1.12500000e+00],[-6.50000000e+00, -3.25000000e+00, -1.25000000e+01,2.22500000e+01],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-6.25000000e+00,  1.36250000e+01, -6.25000000e+00,-1.12500000e+00],[ 9.50000000e+00, -1.15000000e+01,  1.25000000e+01,-1.05000000e+01],[ 9.50000000e+00, -1.15000000e+01,  1.25000000e+01,-1.05000000e+01],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[ 1.25000000e+01, -2.72500000e+01,  1.25000000e+01,2.25000000e+00],[-6.25000000e+00,  1.36250000e+01, -6.25000000e+00,-1.12500000e+00],[-6.50000000e+00, -5.25000000e+00, -1.25000000e+01,2.42500000e+01],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-1.25000000e+01,  2.72500000e+01, -1.25000000e+01,-2.25000000e+00],[-6.25000000e+00,  1.36250000e+01, -6.25000000e+00,-1.12500000e+00],[ 6.25000000e+00, -1.36250000e+01,  6.25000000e+00,1.12500000e+00],[ 0.00000000e+00,  7.53644380e-15,  7.53644380e-15,-1.50728876e-14],[ 0.00000000e+00,  2.51214793e-15,  2.51214793e-15,-5.02429587e-15],[ 0.00000000e+00, -5.02429587e-15, -5.02429587e-15,1.00485917e-14],[ 3.25000000e+00, -1.00000000e+01,  3.25000000e+00,3.50000000e+00],[ 6.28036983e-16, -1.88411095e-15, -6.28036983e-16,1.88411095e-15],[ 0.00000000e+00,  0.00000000e+00,  3.14018492e-16,-3.14018492e-16],[ 3.25000000e+00, -1.00000000e+01,  3.25000000e+00,3.50000000e+00],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-1.12500000e+00, -1.87500000e+00, -1.12500000e+00,4.12500000e+00],[ 1.96261557e-17,  5.88784672e-16,  6.28036983e-16,-1.23644781e-15],[ 3.92523115e-17,  5.88784672e-16,  1.25607397e-15,-1.88411095e-15],[-4.37500000e+00,  8.75000000e+00, -4.37500000e+00,2.66453526e-15],[ 0.00000000e+00,  0.00000000e+00, -6.28036983e-16,6.28036983e-16],[ 6.28036983e-16,  1.37500000e+01,  3.00000000e+00,-1.67500000e+01],[-3.00000000e+00,  1.67500000e+01, -5.02429587e-15,-1.37500000e+01],[ 7.53644380e-15, -1.25607397e-14, -2.51214793e-15,7.53644380e-15],[ 2.51214793e-15, -2.51214793e-15, -2.51214793e-15,2.51214793e-15],[-5.02429587e-15,  1.50728876e-14,  7.53644380e-15,-1.75850355e-14],[ 8.75000000e-01,  6.50000000e+00,  3.87500000e+00,-1.12500000e+01],[ 8.25000000e+00, -8.25000000e+00,  1.12500000e+01,-1.12500000e+01],[ 0.00000000e+00,  2.51214793e-15,  5.02429587e-15,-7.53644380e-15],[-2.51214793e-15,  6.28036983e-15,  5.02429587e-15,-8.79251777e-15],[ 0.00000000e+00, -2.51214793e-15, -2.51214793e-15,5.02429587e-15],[ 6.25000000e-01, -8.00000000e+00,  6.25000000e-01,6.75000000e+00],[ 0.00000000e+00, -4.71027738e-16, -7.26167762e-16,1.19719550e-15],[-3.14018492e-16,  9.42055475e-16,  0.00000000e+00,-6.28036983e-16],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 7.85046229e-16, -7.85046229e-16,  1.57009246e-16,-1.57009246e-16],[ 0.00000000e+00,  4.44089210e-16,  4.44089210e-16,-8.88178420e-16],[ 2.74766180e-16, -2.74766180e-16,  3.14018492e-16,-3.14018492e-16],[-9.42055475e-16,  1.25607397e-15, -3.14018492e-16,0.00000000e+00],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-1.57009246e-16, -1.57009246e-16, -3.14018492e-16,6.28036983e-16],[ 4.44089210e-16, -1.77635684e-15, -1.33226763e-15,2.66453526e-15],[-7.85046229e-17, -2.35513869e-16, -3.14018492e-16,6.28036983e-16],[ 9.42055475e-16, -2.19812944e-15, -1.25607397e-15,2.51214793e-15],[ 6.25000000e-01, -1.25000000e+00,  6.25000000e-01,-6.10622664e-16],[ 0.00000000e+00,  6.28036983e-16,  0.00000000e+00,-6.28036983e-16],[ 8.88178420e-16, -3.10862447e-15, -2.22044605e-15,4.44089210e-15],[-1.25607397e-15,  1.88411095e-15,  6.28036983e-16,-1.25607397e-15],[ 1.25607397e-15, -3.76822190e-15, -2.51214793e-15,5.02429587e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 1.25607397e-15, -3.76822190e-15, -2.51214793e-15,5.02429587e-15],[ 1.33226763e-15, -2.66453526e-15, -1.33226763e-15,2.66453526e-15],[ 1.25607397e-15, -3.76822190e-15, -2.51214793e-15,5.02429587e-15],[ 1.25607397e-15, -1.25607397e-15,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00, -8.88178420e-16, -2.22044605e-15,3.10862447e-15],[ 1.25607397e-15, -1.25607397e-15,  0.00000000e+00,0.00000000e+00],[ 6.25000000e-01,  7.50000000e+00,  1.12500000e+00,-9.25000000e+00],[-3.14018492e-16,  1.88411095e-15,  2.51214793e-15,-4.08224039e-15],[-1.25607397e-15,  1.88411095e-15,  0.00000000e+00,-6.28036983e-16],[-5.00000000e-01,  1.12500000e+01,  0.00000000e+00,-1.07500000e+01],[-1.25607397e-15,  1.25607397e-15,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00, -1.77635684e-15,  0.00000000e+00,1.77635684e-15],[ 0.00000000e+00, -1.25607397e-15, -2.51214793e-15,3.76822190e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 8.88178420e-16, -8.88178420e-16,  0.00000000e+00,0.00000000e+00],[ 1.25607397e-15, -1.25607397e-15,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  1.77635684e-15,  0.00000000e+00,-1.77635684e-15],[ 0.00000000e+00, -1.25607397e-15, -2.51214793e-15,3.76822190e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 8.88178420e-16, -2.66453526e-15, -8.88178420e-16,2.66453526e-15],[ 1.25607397e-15, -1.25607397e-15, -1.25607397e-15,1.25607397e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 2.51214793e-15, -3.76822190e-15,  0.00000000e+00,1.25607397e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  1.77635684e-15,  8.88178420e-16,-2.66453526e-15],[ 0.00000000e+00,  0.00000000e+00,  1.25607397e-15,-1.25607397e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-2.51214793e-15,  3.76822190e-15,  0.00000000e+00,-1.25607397e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00, -1.77635684e-15, -1.77635684e-15,3.55271368e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 1.01250000e+01, -9.00000000e+00,  1.06250000e+01,-1.17500000e+01],[ 6.28036983e-16, -6.28036983e-16,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  2.51214793e-15,  2.51214793e-15,-5.02429587e-15],[ 1.32500000e+01, -1.32500000e+01,  1.37500000e+01,-1.37500000e+01],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00, -2.51214793e-15, -5.02429587e-15,7.53644380e-15],[ 1.77635684e-15, -3.55271368e-15, -1.77635684e-15,3.55271368e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 1.77635684e-15, -1.77635684e-15, -2.66453526e-15,2.66453526e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 1.77635684e-15, -1.77635684e-15,  1.77635684e-15,-1.77635684e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 7.10542736e-15, -1.68753900e-14, -2.66453526e-15,1.24344979e-14],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  2.51214793e-15,  0.00000000e+00,-2.51214793e-15],[ 0.00000000e+00,  1.77635684e-15,  0.00000000e+00,-1.77635684e-15],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[-4.44089210e-15,  8.88178420e-15,  9.76996262e-15,-1.42108547e-14],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  0.00000000e+00,  2.51214793e-15,-2.51214793e-15],[-1.77635684e-15,  1.77635684e-15,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00],[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,0.00000000e+00]])}]]